# 02_baseline_tfidf_logistic_regression.ipynb
##Baseline: TF-IDF + Regressão Logística

#Objetivo:

Construir um modelo baseline para a tarefa de classificação de fake news, estabelecendo uma referência inicial de desempenho e permitindo compreender o comportamento do problema antes da aplicação de modelos mais robustos.

# O que ele faz:

> 1) Pré-processamento dos dados:

- Construção do campo content = title + text
- Remoção de duplicatas completas
- Divisão estratificada em treino e validação

> 2) Extração de features:

- Representação textual baseada em TF-IDF de n-grams de palavras (1–3)

> 3)Treinamento do modelo de Regressão Logística:

- Otimização do threshold com base nas probabilidades (predict_proba) para maximizar o F1-score

> 4) Avaliação do desempenho:

- F1-score no conjunto de validação
- Classification report


In [1]:
# BASELINE: CLASSIFICAÇÃO DE FAKE NEWS
# TF-IDF + Regressão Logística

# 0) Importação das bibliotecas e carregamento dos dados

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

import joblib
import os

train_path = 'train.csv'
test_path = 'test.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

print("Shape treino:", train.shape)
print("Shape teste:", test.shape)


Shape treino: (22844, 6)
Shape teste: (5712, 5)


In [2]:
# 1) Pré-processamento dos dados

# Título e texto Concatenados para fornecer mais informação ao modelo
train['content'] = train['title'].fillna('') + " " + train['text'].fillna('')
test['content'] = test['title'].fillna('') + " " + test['text'].fillna('')

# Remoção duplicatas
train = train.drop_duplicates(subset=["title", "text"]).reset_index(drop=True)


# Definição das variáveis
X = train['content']
y = train['label']


# Divisão treino e validação

# Divisão estratificada para manter proporção das classes
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


# 2) Extração de features

# criação do pipeline (TF-IDF + LogReg)
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words="english",
        ngram_range=(1, 3),
        max_features= 70000,
        min_df=2,
        sublinear_tf=True
    )),
    ("clf", LogisticRegression(
        max_iter=2000,
        class_weight="balanced",
        C= 3.0,
        n_jobs=-1
    ))
])


# 3) Treinamento do modelo

pipeline.fit(X_train, y_train)


#ajustar o threshold
val_proba = pipeline.predict_proba(X_val)[:, 1]

thresholds = np.linspace(0.1, 0.9, 81)
best_t = 0.5
best_f1 = -1

for t in thresholds:
    val_pred_t = (val_proba >= t).astype(int)
    f1 = f1_score(y_val, val_pred_t)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t


# 4) Avaliação de desempenho

# F1-score no conjunto de validação
print(f"Melhor threshold (val): {best_t:.3f}")
print(f"F1 (val) no melhor threshold: {best_f1:.6f}")

Melhor threshold (val): 0.400
F1 (val) no melhor threshold: 0.986977


In [3]:
# Relatório de classificação no threshold ótimo
val_pred_best = (val_proba >= best_t).astype(int)
print("\nClassification report (val):")
print(classification_report(y_val, val_pred_best, digits=4))


Classification report (val):
              precision    recall  f1-score   support

           0     0.9976    0.9941    0.9959      3400
           1     0.9815    0.9925    0.9870      1069

    accuracy                         0.9937      4469
   macro avg     0.9896    0.9933    0.9914      4469
weighted avg     0.9938    0.9937    0.9937      4469



In [4]:
# Refit em todo o treino
pipeline.fit(X, y)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=70000, min_df=2,
                                 ngram_range=(1, 3), stop_words='english',
                                 sublinear_tf=True)),
                ('clf',
                 LogisticRegression(C=3.0, class_weight='balanced',
                                    max_iter=2000, n_jobs=-1))])

In [5]:
# Salvamento do modelo treinado (artifact)
# Permite reprodutibilidade e geração de inferência sem novo treinamento

os.makedirs("artifacts", exist_ok=True)

joblib.dump(
    {
        "pipeline": pipeline,
        "best_threshold": best_t
    },
    "baseline_tfidf_logreg.joblib"
)

print("Artifact salvo em: baseline_tfidf_logreg.joblib")

Artifact salvo em: baseline_tfidf_logreg.joblib


In [6]:
#Inferência Final: Gera a submissão para o Kaggle utilizando o limiar (threshold) de classificação ótimo.

# Score no teste
test_proba = pipeline.predict_proba(test["content"])[:, 1]
test_pred = (test_proba >= best_t).astype(int)


submission = pd.DataFrame({
    'id': test['id'],
    'target': test_pred
})

submission.to_csv('submission_baseline.csv', index=False)

print(f"Submission salva em: submission_baseline.csv")

Submission salva em: submission_baseline.csv


In [7]:
# VERIFICAÇÃO FINAL

print(submission.head())
print("Número de linhas:", submission.shape[0])

      id  target
0   5398       1
1   5503       1
2  23151       0
3  12669       0
4  27864       0
Número de linhas: 5712


# Conclusão

O modelo baseline baseado em TF-IDF de palavras e Regressão Logística apresentou bom desempenho na tarefa de classificação de fake news, alcançando F1-score de 0.986977 no conjunto de validação após ajuste do threshold de decisão (0.400). O resultado confirma a eficácia de abordagens clássicas de representação textual como ponto de partida para o problema.

Apesar do desempenho elevado, a utilização exclusiva de n-grams de palavras pode limitar a captura de padrões estilísticos mais complexos, motivando a exploração de um modelo mais robusto nas etapas seguintes.